# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csvpath = "../WeatherPy/output_data/cities.csv"
citydf = pd.read_csv(csvpath)
citydf

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,vila franca do campo,37.7167,-25.4333,57.20,69,100,4.05,PT,1644811154
1,mataura,-46.1927,168.8643,70.90,39,100,5.77,NZ,1644810637
2,arraial do cabo,-22.9661,-42.0278,75.07,93,9,15.28,BR,1644810717
3,carnarvon,-24.8667,113.6333,86.07,74,6,12.66,AU,1644810639
4,skjervoy,70.0311,20.9714,32.14,61,100,22.17,NO,1644811154
...,...,...,...,...,...,...,...,...,...
574,cascais,38.6979,-9.4215,53.51,71,0,20.00,PT,1644812015
575,amahai,-3.3333,128.9167,80.28,78,100,2.48,ID,1644812016
576,lenger,42.1830,69.8839,35.04,98,100,15.05,KZ,1644812016
577,mitsamiouli,-11.3847,43.2844,80.46,79,37,2.35,KM,1644812016


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps with API key
gmaps.configure(api_key=g_key)

#Store "Lat" and "Lng" into locations
locations = citydf[["Lat", "Lng"]].astype(float)

#Convert Humidity to float and store
humidity = citydf["Humidity"].astype(float)

In [4]:
#Create humidity Heatmap layer
fig = gmaps.figure(center=[20,0], zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 2)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
idealtemp = citydf.loc[citydf["Max Temp"] > 75]
besttemp = idealtemp.loc[idealtemp["Max Temp"] < 85]
nowind = besttemp.loc[besttemp["Wind Speed"] < 10]
sunny = nowind.loc[nowind["Cloudiness"] < 10]
sunny

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
36,bubaque,11.2833,-15.8333,77.95,59,0,4.09,GW,1644811161
92,ashti,19.3833,76.2333,75.63,22,9,8.32,IN,1644811295
189,shwebo,22.5667,95.7000,75.85,31,0,2.77,MM,1644811438
266,kavaratti,10.5669,72.6420,80.94,70,2,8.41,IN,1644811517
300,champerico,14.3000,-91.9167,83.35,72,4,3.76,GT,1644811585
457,barra patuca,15.8000,-84.2833,75.25,84,1,8.61,HN,1644811806
479,betioky,-23.7167,44.3833,77.02,56,0,8.48,MG,1644811871
555,acajutla,13.5928,-89.8275,81.81,60,0,9.31,SV,1644811830


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = sunny
hotel_df["Hotel Name"] = ""

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"radius": 5000, "type": "lodging", "key": g_key}

for index, row in hotel_df.iterrows():
    try:
        lat = row["Lat"]
        lon = row["Lng"]
        location = f"{lat}, {lon}"
        params["location"] = location
        response = requests.get(base_url, params=params).json()
        results = response["results"]
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except:
        print("There are no hotels within 5000m of " + row["City"] + ".")
    
hotel_df

C:\Users\Clare\AppData\Local\Temp/ipykernel_11376/2522082899.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""
C:\Users\Clare\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


There are no hotels within 5000m of barra patuca.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
36,bubaque,11.2833,-15.8333,77.95,59,0,4.09,GW,1644811161,"Casa Dora, Bubaque"
92,ashti,19.3833,76.2333,75.63,22,9,8.32,IN,1644811295,Sree Sant Savta Mali
189,shwebo,22.5667,95.7000,75.85,31,0,2.77,MM,1644811438,MOON SHINE Guest House
266,kavaratti,10.5669,72.6420,80.94,70,2,8.41,IN,1644811517,Hotels in Lakshadweep Islands
300,champerico,14.3000,-91.9167,83.35,72,4,3.76,GT,1644811585,"Hotel y restaurante ""El Diamante"""
457,barra patuca,15.8000,-84.2833,75.25,84,1,8.61,HN,1644811806,
479,betioky,-23.7167,44.3833,77.02,56,0,8.48,MG,1644811871,Bungalows Acacias Hotel
555,acajutla,13.5928,-89.8275,81.81,60,0,9.31,SV,1644811830,Laureles


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
fig = gmaps.figure(center=[20,0], zoom_level=2)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))